# Am building the Rag application
## Data Ingestion 

In [4]:
from langchain_community.document_loaders import TextLoader
loader=TextLoader('devops.txt')
text=loader.load()
print(text)

[Document(metadata={'source': 'devops.txt'}, page_content='Why DevOps?\nIntroduction\n\nIn today’s fast-paced digital world, the demand for rapid software delivery, higher reliability, and seamless user experience is greater than ever. DevOps has emerged as a transformative approach to software development and IT operations that bridges the gap between traditionally siloed teams — development and operations. It is not just a set of tools but a culture and a practice that fosters collaboration, automation, and continuous improvement.\n\nThe Origins and Purpose of DevOps\n\nDevOps originated as a response to the inefficiencies in the software development lifecycle, where development teams would build features, and operations teams would deploy and maintain them. The disconnect often led to delays, miscommunications, and a slow feedback loop, which negatively impacted the business’s agility and innovation.\n\nDevOps integrates these functions by emphasizing:\n\nCollaboration and communica

In [9]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('sops/Interview-questions.pdf')
doc=loader.load()
doc[0]

Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': 'sops/Interview-questions.pdf', 'total_pages': 81, 'page': 0, 'page_label': '1'}, page_content='General DevOps \n1. Question: What is DevOps?  \nAnswer: DevOps is a set of practices that combines software development \n(Dev) and IT operations (Ops) to shorten the systems development life cycle \nand provide continuous delivery with high software quality. \n2. Question: Explain the core principles of DevOps.  \nAnswer: The core principles of DevOps include: \no Automation: Automating repetitive tasks to increase efficiency and reduce \nhuman errors. \no Continuous Integration and Continuous Delivery (CI/CD): Ensuring code \nchanges are automatically tested and deployed to production. \no Collaboration and Communication: Enhancing communication between \ndevelopment and operations teams. \no Infrastructure as Code (IaC): Managing infrastructure using code and \nsoftware development techniques. \no M

In [11]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
loader=PyPDFDirectoryLoader('sops')
docs=loader.load()
docs[0]

Ignoring wrong pointing object 6 0 (offset 0)


Document(metadata={'producer': 'macOS Version 15.6.1 (Build 24G90) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20251012073155Z00'00'", 'moddate': "D:20251012073155Z00'00'", 'source': 'sops/K8-ERRORs.pdf', 'total_pages': 606, 'page': 0, 'page_label': '1'}, page_content='## Scenario #1: Zombie Pods Causing NodeDrain to Hang  - Category: Cluster Management  - Environment: K8s v1.23, On-prem bare metal, Systemd cgroups   Scenario  - Summary: Node drain stuck indeﬁnitely due to unresponsive terminating pod.  - What Happened:A pod with a custom ﬁnalizer never completed termination, blocking kubectl drain. Even after the pod was marked for deletion, the API server kept waiting because the ﬁnalizer wasn’t removed.  - Diagnosis Steps:    • Checked kubectl get pods --all-namespaces -o wide to ﬁnd lingering pods.   • Found pod stuck in Terminating state for over 20 minutes.   • Used kubectl describe pod <pod> to identify the presence of a custom ﬁnalizer.   • Investigated controlle

# Data Transformation

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_docs=text_splitter.split_documents(docs)
final_docs[0]

Document(metadata={'producer': 'macOS Version 15.6.1 (Build 24G90) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20251012073155Z00'00'", 'moddate': "D:20251012073155Z00'00'", 'source': 'sops/K8-ERRORs.pdf', 'total_pages': 606, 'page': 0, 'page_label': '1'}, page_content='## Scenario #1: Zombie Pods Causing NodeDrain to Hang  - Category: Cluster Management  - Environment: K8s v1.23, On-prem bare metal, Systemd cgroups   Scenario  - Summary: Node drain stuck indeﬁnitely due to unresponsive terminating pod.  - What Happened:A pod with a custom ﬁnalizer never completed termination, blocking kubectl drain. Even after the pod was marked for deletion, the API server kept waiting because the ﬁnalizer wasn’t removed.  - Diagnosis Steps:    • Checked kubectl get pods --all-namespaces -o wide to ﬁnd lingering pods.   • Found pod stuck in Terminating state for over 20 minutes.   • Used kubectl describe pod <pod> to identify the presence of a custom ﬁnalizer.   • Investigated controlle

#### chunk_size=6,chunk overlap =3
manoj is devops engineer and he 
is working on python

## Embeddings and Vector store

In [23]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')

from langchain_openai.embeddings import OpenAIEmbeddings
embeddings=OpenAIEmbeddings(model='text-embedding-3-small')
embeddings


OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x117d0d0d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x11996e420>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [27]:
query="this is to test"
result=embeddings.embed_query('query')
result[0]

-0.008966981433331966

In [28]:
len(result)

1536

In [30]:
## emmeddings vector store

from langchain_chroma import Chroma
db=Chroma.from_documents(final_docs,embeddings)
db


## Data Retrival

In [32]:
query="What is DevOps?"
result=db.similarity_search(query)
result

[Document(id='2a0b3d10-8f39-43f1-8713-483d3dc3b61e', metadata={'page_label': '1', 'producer': 'PyPDF', 'total_pages': 81, 'page': 0, 'creator': 'PyPDF', 'creationdate': '', 'source': 'sops/Interview-questions.pdf'}, page_content='General DevOps \n1. Question: What is DevOps?  \nAnswer: DevOps is a set of practices that combines software development \n(Dev) and IT operations (Ops) to shorten the systems development life cycle \nand provide continuous delivery with high software quality. \n2. Question: Explain the core principles of DevOps.  \nAnswer: The core principles of DevOps include: \no Automation: Automating repetitive tasks to increase efficiency and reduce \nhuman errors. \no Continuous Integration and Continuous Delivery (CI/CD): Ensuring code \nchanges are automatically tested and deployed to production. \no Collaboration and Communication: Enhancing communication between \ndevelopment and operations teams. \no Infrastructure as Code (IaC): Managing infrastructure using code 

## LLM communication


In [35]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT']=os.getenv('LANGCHAIN_PROJECT')
os.environ['LANGSMITH_TRACING']="true"

In [36]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-5")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x11a9b3a70>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11a7f43e0>, root_client=<openai.OpenAI object at 0x11aa92ea0>, root_async_client=<openai.AsyncOpenAI object at 0x11aa07410>, model_name='gpt-5', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [38]:
query="Hello gpt-5 how are you"
result=llm.invoke(query)
result

AIMessage(content='Hi! I’m doing well, thanks for asking. How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 155, 'prompt_tokens': 14, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CSSEe8Ekir67069LT82mqZ8pArw1m', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a6d0cd5f-87d5-418e-b6a0-715f69644177-0', usage_metadata={'input_tokens': 14, 'output_tokens': 155, 'total_tokens': 169, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 128}})

In [39]:
type(result)

langchain_core.messages.ai.AIMessage

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompts=ChatPromptTemplate.from_messages(
    [
        ('system','Hey you are an AI expert answer my question'),
        ('user','{input}')
    ]
)

prompts



ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hey you are an AI expert answer my question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [42]:
query="what is GenAI"
chain=prompts|llm
result=chain.invoke(query)
result



AIMessage(content='GenAI (Generative AI) is a class of artificial intelligence systems that can create new content—text, images, code, audio, video, and more—based on patterns learned from large datasets.\n\nHow it works (high level):\n- Trains large neural networks on vast data using self‑supervised learning.\n- Text models (LLMs) predict the next token in a sequence; image/audio/video models often use diffusion or autoregressive decoding to generate from noise or prompts.\n- Instruction tuning and RLHF make models follow user prompts; retrieval (RAG) can ground answers in external sources.\n\nWhat it can do:\n- Text: drafting, summarization, translation, Q&A, chat, analytics, code generation.\n- Media: create/edit images, audio, and video; design prototypes; generate marketing assets.\n- Data tasks: extract structure from documents, generate synthetic data, classify or tag content.\n\nCommon examples:\n- Text: GPT-4/4o, Claude, Llama.\n- Images: DALL·E, Midjourney, Stable Diffusion.\

In [43]:
from langchain_core.output_parsers import StrOutputParser
output_parser=StrOutputParser()

chain=prompts|llm|output_parser
query="what is GenAI"
result=chain.invoke(query)
result


'GenAI (generative AI) is a type of artificial intelligence that creates new content—text, images, audio, video, or code—by learning patterns from large datasets and then generating outputs that resemble what it has learned.\n\nKey points:\n- How it works: Models learn to predict the next token (text/code) or to turn noise into images/video (diffusion). Common architectures include transformers (LLMs) and diffusion models. They can be adapted with fine-tuning or retrieval-augmented generation (RAG).\n- Examples: ChatGPT/Claude/Gemini/Llama (text), DALL·E/Stable Diffusion/Midjourney (images), Sora (video), GitHub Copilot (code), various speech and music models.\n- What it’s used for: Drafting and summarizing text, answering questions, coding assistance, design and marketing content, data exploration, customer support, and rapid prototyping.\n- Benefits: Speed, scale, versatility, and natural language interfaces.\n- Limitations/risks: Hallucinations (incorrect outputs), bias, privacy/IP 